## 1. Load JSON Dataset
### COVID-19 Open Research Dataset Challenge (CORD-19) Working Notebooks

This is a working notebook for the [COVID-19 Open Research Dataset Challenge (CORD-19)](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge) to help you jump start your analysis of the CORD-19 dataset.  

<img src="https://miro.medium.com/max/3648/1*596Ur1UdO-fzQsaiGPrNQg.png" width="900"/>

Attributions:
* The licenses for each dataset used for this workbook can be found in the *all _ sources _ metadata csv file* which is included in the [downloaded dataset](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge/download).  
* For the 2020-03-03 dataset: 
  * `comm_use_subset`: Commercial use subset (includes PMC content) -- 9000 papers, 186Mb
  * `noncomm_use_subset`: Non-commercial use subset (includes PMC content) -- 1973 papers, 36Mb
  * `biorxiov_medrxiv`: bioRxiv/medRxiv subset (pre-prints that are not peer reviewed) -- 803 papers, 13Mb
* When using Databricks or Databricks Community Edition, a copy of this dataset has been made available at `/databricks-datasets/COVID/CORD-19`
* This notebook is freely available to share, licensed under [CC BY 3.0](https://creativecommons.org/licenses/by/3.0/us/)

#### Configure Path Variables
You can find the CORD-19 (2020-03-13) dataset available at: `/databricks-datasets/COVID/CORD-19/2020-03-13/`

In [3]:
# Configure Paths in Python
comm_use_subset_path = "/databricks-datasets/COVID/CORD-19/2020-03-13/comm_use_subset/comm_use_subset/"
noncomm_use_subset_path = "/databricks-datasets/COVID/CORD-19/2020-03-13/noncomm_use_subset/noncomm_use_subset/"
biorxiv_medrxiv_path = "/databricks-datasets/COVID/CORD-19/2020-03-13/biorxiv_medrxiv/biorxiv_medrxiv/"
json_schema_path = "/databricks-datasets/COVID/CORD-19/2020-03-13/json_schema.txt"

In [4]:
# Configure Path as Shell Enviroment Variables
import os
os.environ['comm_use_subset_path']=''.join(comm_use_subset_path)
os.environ['noncomm_use_subset_path']=''.join(noncomm_use_subset_path)
os.environ['biorxiv_medrxiv_path']=''.join(biorxiv_medrxiv_path)
os.environ['json_schema_path']=''.join(json_schema_path)

#### Review JSON Schema
The schema for these datasets is defined by `json_schema.txt` as noted in the following cell.

In [6]:
%sh 
cat /dbfs$json_schema_path

# JSON schema of full text documents


{
 "paper_id": <str>, # 40-character sha1 of the PDF
 "metadata": {
 "title": <str>,
 "authors": [ # list of author dicts, in order
 {
 "first": <str>,
 "middle": <list of str>,
 "last": <str>,
 "suffix": <str>,
 "affiliation": <dict>,
 "email": <str>
 },
 ...
 ],
 "abstract": [ # list of paragraphs in the abstract
 {
 "text": <str>,
 "cite_spans": [ # list of character indices of inline citations
 # e.g. citation "[7]" occurs at positions 151-154 in "text"
 # linked to bibliography entry BIBREF3
 {
 "start": 151,
 "end": 154,
 "text": "[7]",
 "ref_id": "BIBREF3"
 },
 ...
 ],
 "ref_spans": <list of dicts similar to cite_spans>, # e.g. inline reference to "Table 1"
 "section": "Abstract"
 },
 ...
 ],
 "body_text": [ # list of paragraphs in full body
 # paragraph dicts look the same as above
 {
 "text": <str>,
 "cite_spans": [],
 "ref_spans": [],
 "eq_spans": [],
 "section": "Introduction"
 },
 ...
 {
 ...,
 "section": "Conclusion"
 }
 ],
 "bib_entries": {
 "BIBREF0": {
 "ref_id": <str>,
 "title": <str>,
 "authors": <list of dict> # same structure as earlier,
 # but without `affiliation` or `email`
 "year": <int>,
 "venue": <str>,
 "volume": <str>,
 "issn": <str>,
 "pages": <str>,
 "other_ids": {
 "DOI": [
 <str>
 ]
 }
 },
 "BIBREF1": {},
 ...
 "BIBREF25": {}
 },
 "ref_entries":
 "FIGREF0": {
 "text": <str>, # figure caption text
 "type": "figure"
 },
 ...
 "TABREF13": {
 "text": <str>, # table caption text
 "type": "table"
 }
 },
 "back_matter": <list of dict> # same structure as body_text
 }
}

In [7]:
%fs ls /tmp/dennylee/

path,name,size
dbfs:/tmp/dennylee/COVID/,COVID/,0
dbfs:/tmp/dennylee/samples/,samples/,0


#### Configure Parquet Path Variables
Save the data in Parquet format at: `/tmp/dennylee/COVID/CORD-19/2020-03-13/`

In [9]:
# Configure Parquet Paths in Python
comm_use_subset_pq_path = "/tmp/dennylee/COVID/CORD-19/2020-03-13/comm_use_subset.parquet"
noncomm_use_subset_pq_path = "/tmp/dennylee/COVID/CORD-19/2020-03-13/noncomm_use_subset.parquet"
biorxiv_medrxiv_pq_path = "/tmp/dennylee/COVID/CORD-19/2020-03-13/biorxiv_medrxiv/biorxiv_medrxiv.parquet"

# Configure Path as Shell Enviroment Variables
os.environ['comm_use_subset_pq_path']=''.join(comm_use_subset_pq_path)
os.environ['noncomm_use_subset_pq_path']=''.join(noncomm_use_subset_pq_path)
os.environ['biorxiv_medrxiv_pq_path']=''.join(biorxiv_medrxiv_pq_path)

#### Read comm_use_subset JSON Files
As these are correctly formed JSON files, you can use `spark.read.json` to read these files.  Note, you will need to specify the *multiline* option.

In [11]:
# comm_use_subset
comm_use_subset = spark.read.option("multiLine", True).json(comm_use_subset_path)
comm_use_subset.printSchema()

root
-- abstract: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- cite_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- ref_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- section: string (nullable = true)
 | |-- text: string (nullable = true)
-- back_matter: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- cite_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- ref_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- section: string (nullable = true)
 | |-- text: string (nullable = true)
-- bib_entries: struct (nullable = true)
 |-- BIBREF0: struct (nullable = true)
 | |-- authors: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- first: string (nullable = true)
 | | | |-- last: string (nullable = true)
 | | | |-- middle: array (nullable = true)
 | | | | |-- element: string (containsNull = true)
 | | | |-- suffix: string (nullable = true)
 | |-- issn: string (nullable = true)
 | |-- other_ids: struct (nullable = true)
 | | |-- DOI: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMCID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | |-- pages: string (nullable = true)
 | |-- ref_id: string (nullable = true)
 | |-- title: string (nullable = true)
 | |-- venue: string (nullable = true)
 | |-- volume: string (nullable = true)
 | |-- year: long (nullable = true)
 |-- BIBREF1: struct (nullable = true)
 | |-- authors: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- first: string (nullable = true)
 | | | |-- last: string (nullable = true)
 | | | |-- middle: array (nullable = true)
 | | | | |-- element: string (containsNull = true)
 | | | |-- suffix: string (nullable = true)
 | |-- issn: string (nullable = true)
 | |-- other_ids: struct (nullable = true)
 | | |-- DOI: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMCID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- arXiv: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | |-- pages: string (nullable = true)
 | |-- ref_id: string (nullable = true)
 | |-- title: string (nullable = true)
 | |-- venue: string (nullable = true)
 | |-- volume: string (nullable = true)
 | |-- year: long (nullable = true)
 |-- BIBREF10: struct (nullable = true)
 | |-- authors: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- first: string (nullable = true)
 | | | |-- last: string (nullable = true)
 | | | |-- middle: array (nullable = true)
 | | | | |-- element: string (containsNull = true)
 | | | |-- suffix: string (nullable = true)
 | |-- issn: string (nullable = true)
 | |-- other_ids: struct (nullable = true)
 | | |-- DOI: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMCID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMID: array (nullable = true)
 | | | |-- element: st

In [12]:
# Number of records (originally number of JSON documents)
comm_use_subset.count()

Out[45]: 9000

##### Validate number of files

In [14]:
%sh 
ls /dbfs$comm_use_subset_path | wc -l

9000

#### Save comm_use_subset JSON Files
It takes awhile to read the files, so let's save them as Parquet format to improve query performance

In [16]:
# Get number of partitions
comm_use_subset.rdd.getNumPartitions()

Out[47]: 286

In [17]:
# Write out in Pqarquet format in four partitions 
# Note, this cluster has four nodes 
comm_use_subset.repartition(4).write.format("parquet").mode("overwrite").save(comm_use_subset_pq_path)

In [18]:
%sh 
ls -lsgA /dbfs/tmp/dennylee/COVID/CORD-19/2020-03-13/comm_use_subset.parquet

total 265551
 1 -rwxrwxrwx 1 root 428 Mar 19 03:50 _committed_3280191404807126853
67581 -rwxrwxrwx 1 root 69202453 Mar 19 03:50 part-00000-tid-3280191404807126853-870e6efc-4514-4959-9a0a-5c9dfc643a41-4157-1-c000.snappy.parquet
68174 -rwxrwxrwx 1 root 69809276 Mar 19 03:50 part-00001-tid-3280191404807126853-870e6efc-4514-4959-9a0a-5c9dfc643a41-4159-1-c000.snappy.parquet
63269 -rwxrwxrwx 1 root 64787116 Mar 19 03:50 part-00002-tid-3280191404807126853-870e6efc-4514-4959-9a0a-5c9dfc643a41-4160-1-c000.snappy.parquet
66527 -rwxrwxrwx 1 root 68123343 Mar 19 03:50 part-00003-tid-3280191404807126853-870e6efc-4514-4959-9a0a-5c9dfc643a41-4158-1-c000.snappy.parquet
 0 -rwxrwxrwx 1 root 0 Mar 19 03:50 _started_3280191404807126853
 0 -rwxrwxrwx 1 root 0 Mar 19 03:50 _SUCCESS

In [19]:
# Reread files
comm_use_subset = spark.read.format("parquet").load(comm_use_subset_pq_path)

In [20]:
# Number of records (originally number of JSON documents)
comm_use_subset.count()

Out[52]: 9000

#### Read noncomm_use_subset JSON Files
As these are correctly formed JSON files, you can use `spark.read.json` to read these files.  Note, you will need to specify the *multiline* option.

In [22]:
# noncomm_use_subset
noncomm_use_subset = spark.read.option("multiLine", True).json(noncomm_use_subset_path)
noncomm_use_subset.printSchema()

root
-- abstract: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- cite_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- ref_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- section: string (nullable = true)
 | |-- text: string (nullable = true)
-- back_matter: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- cite_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- ref_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- section: string (nullable = true)
 | |-- text: string (nullable = true)
-- bib_entries: struct (nullable = true)
 |-- BIBREF0: struct (nullable = true)
 | |-- authors: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- first: string (nullable = true)
 | | | |-- last: string (nullable = true)
 | | | |-- middle: array (nullable = true)
 | | | | |-- element: string (containsNull = true)
 | | | |-- suffix: string (nullable = true)
 | |-- issn: string (nullable = true)
 | |-- other_ids: struct (nullable = true)
 | | |-- DOI: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | |-- pages: string (nullable = true)
 | |-- ref_id: string (nullable = true)
 | |-- title: string (nullable = true)
 | |-- venue: string (nullable = true)
 | |-- volume: string (nullable = true)
 | |-- year: long (nullable = true)
 |-- BIBREF1: struct (nullable = true)
 | |-- authors: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- first: string (nullable = true)
 | | | |-- last: string (nullable = true)
 | | | |-- middle: array (nullable = true)
 | | | | |-- element: string (containsNull = true)
 | | | |-- suffix: string (nullable = true)
 | |-- issn: string (nullable = true)
 | |-- other_ids: struct (nullable = true)
 | | |-- DOI: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | |-- pages: string (nullable = true)
 | |-- ref_id: string (nullable = true)
 | |-- title: string (nullable = true)
 | |-- venue: string (nullable = true)
 | |-- volume: string (nullable = true)
 | |-- year: long (nullable = true)
 |-- BIBREF10: struct (nullable = true)
 | |-- authors: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- first: string (nullable = true)
 | | | |-- last: string (nullable = true)
 | | | |-- middle: array (nullable = true)
 | | | | |-- element: string (containsNull = true)
 | | | |-- suffix: string (nullable = true)
 | |-- issn: string (nullable = true)
 | |-- other_ids: struct (nullable = true)
 | | |-- DOI: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- arXiv: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | |-- pages: string (nullable = true)
 | |-- ref_id: string (nullable = true)
 | |-- title: string (nullable = true)
 | |-- venue: string (nullable = true)
 | |-- volume: string (nullable = true)
 | |-- year: long (nullable = true)
 |-- BI

In [23]:
# Number of records (originally number of JSON documents)
noncomm_use_subset.count()

Out[54]: 1973

In [24]:
%sh 
ls /dbfs$noncomm_use_subset_path | wc -l

1973

#### Save noncomm_use_subset JSON Files
It takes awhile to read the files, so let's save them as Parquet format to improve query performance

In [26]:
# Get number of partitions
noncomm_use_subset.rdd.getNumPartitions()

Out[56]: 63

In [27]:
# Write out in Parquet format in four partitions 
# Note, this cluster has four nodes 
noncomm_use_subset.repartition(4).write.format("parquet").mode("overwrite").save(noncomm_use_subset_pq_path)

In [28]:
%sh
ls -lsgA /dbfs$noncomm_use_subset_pq_path

total 63914
 1 -rwxrwxrwx 1 root 428 Mar 19 05:39 _committed_3274535761126764333
18190 -rwxrwxrwx 1 root 18626173 Mar 19 05:39 part-00000-tid-3274535761126764333-db98e008-2702-492e-bcb3-54dd0652a8a2-4296-1-c000.snappy.parquet
15861 -rwxrwxrwx 1 root 16241092 Mar 19 05:39 part-00001-tid-3274535761126764333-db98e008-2702-492e-bcb3-54dd0652a8a2-4298-1-c000.snappy.parquet
14123 -rwxrwxrwx 1 root 14461117 Mar 19 05:39 part-00002-tid-3274535761126764333-db98e008-2702-492e-bcb3-54dd0652a8a2-4299-1-c000.snappy.parquet
15740 -rwxrwxrwx 1 root 16117516 Mar 19 05:39 part-00003-tid-3274535761126764333-db98e008-2702-492e-bcb3-54dd0652a8a2-4297-1-c000.snappy.parquet
 0 -rwxrwxrwx 1 root 0 Mar 19 05:39 _started_3274535761126764333
 0 -rwxrwxrwx 1 root 0 Mar 19 05:39 _SUCCESS

In [29]:
# Reread files
noncomm_use_subset = spark.read.format("parquet").load(noncomm_use_subset_pq_path)

In [30]:
# Number of records (originally number of JSON documents)
noncomm_use_subset.count()

Out[60]: 1973

#### Read biorxiv_medrxiv JSON Files
As these are correctly formed JSON files, you can use `spark.read.json` to read these files.  Note, you will need to specify the *multiline* option.

In [32]:
# biorxiv_medrxiv
biorxiv_medrxiv = spark.read.option("multiLine", True).json(biorxiv_medrxiv_path)
biorxiv_medrxiv.count()

Out[61]: 803

In [33]:
# Write out in Parquet format in four partitions 
# Note, this cluster has four nodes 
biorxiv_medrxiv.repartition(4).write.format("parquet").mode("overwrite").save(biorxiv_medrxiv_pq_path)

In [34]:
# Re-read files
biorxiv_medrxiv = spark.read.format("parquet").load(biorxiv_medrxiv_pq_path)
biorxiv_medrxiv.count()

Out[63]: 803

From this point forward, we can read the Parquet files instead of the original JSON files for faster performance.